### Restart and run all cells

#### csv files: sell-alert

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2021, 12, 29)

In [2]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[10]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = trunc(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [3]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}','css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [4]:
sql = '''
SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '%s'
'''
# AND to_price >= fm_price
sql = sql % today
print(sql)


SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '2021-12-29'



In [5]:
df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status
0,BGRIM,42.00,0,"9,000","9,000",40.00,2021-12-21,2021-12-29,6,40.25,40.00,-1,40.75,56.00,54.00,"378,000.00",I
14,CKP,5.50,0,"30,000","30,000",5.00,2021-12-28,2021-12-29,1,5.05,5.00,-1,5.05,6.30,6.44,"165,000.00",I
10,DCC,2.92,12,"40,000","40,000",2.90,2021-12-28,2021-12-29,1,2.82,2.84,1,2.86,3.40,3.42,"116,800.00",I
7,DIF,14.20,0,"10,000","60,000",14.70,2021-12-27,2021-12-29,2,13.90,14.00,1,14.00,14.40,15.80,"142,000.00",I
13,IVL,44.00,12,"3,000","18,000",42.00,2021-12-28,2021-12-29,1,42.75,43.00,1,43.25,50.00,54.38,"132,000.00",I
8,KBANK,143.25,0,800,"2,400",136.50,2021-12-27,2021-12-29,2,138.00,142.00,8,142.50,153.00,170.00,"114,600.00",I
1,NER,7.50,0,"1,200","36,000",7.45,2021-12-22,2021-12-29,5,7.15,7.05,-2,7.20,8.65,10.36,"9,000.00",I
9,PTTGC,58.75,12,"1,000","5,400",66.75,2021-12-27,2021-12-29,2,57.75,58.50,3,58.50,70.00,74.38,"58,750.00",S
11,RCL,51.50,0,"1,500","4,500",43.00,2021-12-28,2021-12-29,1,48.75,49.00,1,49.75,67.75,58.00,"77,250.00",I
4,RJH,34.75,0,"1,500","4,500",33.00,2021-12-27,2021-12-29,2,32.50,33.00,2,33.00,39.00,36.20,"52,125.00",I


In [6]:
my_array = df_stock.to_numpy()
my_array

array([['BGRIM', 42.0, 0, 9000, 9000, 40.0, '2021-12-21', '2021-12-29',
        6, 40.25, 40.0, -1, 40.75, 56.0, 54.0, 378000.0, 'I'],
       ['NER', 7.5, 0, 1200, 36000, 7.45, '2021-12-22', '2021-12-29', 5,
        7.15, 7.05, -2, 7.2, 8.65, 10.36, 9000.0, 'I'],
       ['TSTH', 1.58, 0, 75000, 75000, 1.5, '2021-12-23', '2021-12-29',
        4, 1.48, 1.46, -2, 1.5, 2.44, 1.5, 118500.0, 'I'],
       ['VNT', 39.0, 0, 4500, 4500, 37.5, '2021-12-14', '2021-12-29', 11,
        38.75, 38.5, -1, 38.75, 39.0, 43.5, 175500.0, 'I'],
       ['RJH', 34.75, 0, 1500, 4500, 33.0, '2021-12-27', '2021-12-29', 2,
        32.5, 33.0, 2, 33.0, 39.0, 36.2, 52125.0, 'I'],
       ['SAT', 22.2, 0, 2500, 7500, 21.1, '2021-12-27', '2021-12-29', 2,
        21.2, 21.5, 3, 21.5, 23.5, 26.8, 55500.0, 'I'],
       ['TOP', 52.5, 0, 3000, 3000, 50.0, '2021-12-27', '2021-12-29', 2,
        48.25, 49.75, 6, 49.75, 66.0, 64.33, 157500.0, 'I'],
       ['DIF', 14.2, 0, 10000, 60000, 14.7, '2021-12-27', '2021-12-29',
      

In [7]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ 8,  9, 12,  2,  7,  6, 11,  2,  2,  1,  4, 10, 13,  4, 10])

In [8]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status,spd
0,BGRIM,42.00,0,"9,000","9,000",40.00,2021-12-21,2021-12-29,6,40.25,40.00,-1,40.75,56.00,54.00,"378,000.00",I,8
1,NER,7.50,0,"1,200","36,000",7.45,2021-12-22,2021-12-29,5,7.15,7.05,-2,7.20,8.65,10.36,"9,000.00",I,9
2,TSTH,1.58,0,"75,000","75,000",1.50,2021-12-23,2021-12-29,4,1.48,1.46,-2,1.50,2.44,1.50,"118,500.00",I,12
3,VNT,39.00,0,"4,500","4,500",37.50,2021-12-14,2021-12-29,11,38.75,38.50,-1,38.75,39.00,43.50,"175,500.00",I,2
4,RJH,34.75,0,"1,500","4,500",33.00,2021-12-27,2021-12-29,2,32.50,33.00,2,33.00,39.00,36.20,"52,125.00",I,7
5,SAT,22.20,0,"2,500","7,500",21.10,2021-12-27,2021-12-29,2,21.20,21.50,3,21.50,23.50,26.80,"55,500.00",I,6
6,TOP,52.50,0,"3,000","3,000",50.00,2021-12-27,2021-12-29,2,48.25,49.75,6,49.75,66.00,64.33,"157,500.00",I,11
7,DIF,14.20,0,"10,000","60,000",14.70,2021-12-27,2021-12-29,2,13.90,14.00,1,14.00,14.40,15.80,"142,000.00",I,2
8,KBANK,143.25,0,800,"2,400",136.50,2021-12-27,2021-12-29,2,138.00,142.00,8,142.50,153.00,170.00,"114,600.00",I,2
9,PTTGC,58.75,12,"1,000","5,400",66.75,2021-12-27,2021-12-29,2,57.75,58.50,3,58.50,70.00,74.38,"58,750.00",S,1


In [9]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt, dividend,
volbuy * dividend AS div_amt, (volbuy * dividend)/(volbuy * price) * 100 AS cost_pct, period
FROM buy
WHERE period IN ("1","2","3","4")
'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
10,BCH,"15,000",21.30,"319,500.00",0.33,"4,950.00",1.55,3
29,BGRIM,"9,000",40.00,"360,000.00",0.45,"4,050.00",1.12,4
19,CKP,"30,000",5.00,"150,000.00",0.04,"1,050.00",0.70,3
28,DCC,"40,000",2.90,"116,000.00",0.18,"7,360.00",6.34,4
6,DIF,"60,000",14.70,"882,000.00",1.04,"62,640.00",7.10,2
12,DOHOME,"8,000",25.00,"200,000.00",0.10,800.00,0.40,3
25,EPG,"14,000",11.20,"156,800.00",0.23,"3,220.00",2.05,3
8,GLOBAL,"7,500",20.00,"150,000.00",nan,nan,nan,3
17,HREIT,"30,000",8.95,"268,500.00",0.69,"20,796.00",7.75,2
27,IMH,"9,000",11.00,"99,000.00",nan,nan,nan,3


In [10]:
df_merge = pd.merge(df_stock, df_buy, on=['name'], how='inner')
cols = 'name spd target sh qty available_qty cost fm_price to_price ttl max max52 css_price status period amount'.split()
df_merge[cols].style.format(format_dict)

,name,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
0,BGRIM,8,42.00,0,"9,000","9,000",40.00,40.25,40.00,-1,40.75,56.00,54.00,I,4,"378,000.00"
1,NER,9,7.50,0,"1,200","36,000",7.45,7.15,7.05,-2,7.20,8.65,10.36,I,4,"9,000.00"
2,TSTH,12,1.58,0,"75,000","75,000",1.50,1.48,1.46,-2,1.50,2.44,1.50,I,3,"118,500.00"
3,VNT,2,39.00,0,"4,500","4,500",37.50,38.75,38.50,-1,38.75,39.00,43.50,I,4,"175,500.00"
4,RJH,7,34.75,0,"1,500","4,500",33.00,32.50,33.00,2,33.00,39.00,36.20,I,4,"52,125.00"
5,SAT,6,22.20,0,"2,500","7,500",21.10,21.20,21.50,3,21.50,23.50,26.80,I,4,"55,500.00"
6,TOP,11,52.50,0,"3,000","3,000",50.00,48.25,49.75,6,49.75,66.00,64.33,I,4,"157,500.00"
7,DIF,2,14.20,0,"10,000","60,000",14.70,13.90,14.00,1,14.00,14.40,15.80,I,2,"142,000.00"
8,KBANK,2,143.25,0,800,"2,400",136.50,138.00,142.00,8,142.50,153.00,170.00,I,4,"114,600.00"
9,PTTGC,1,58.75,12,"1,000","5,400",66.75,57.75,58.50,3,58.50,70.00,74.38,S,1,"58,750.00"


In [11]:
sell_alert = df_merge[cols]
sell_alert.set_index(['name'],inplace=True)
sell_alert.sort_values(['spd'],ascending=[True])

,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
name,,,,,,,,,,,,,,,
PTTGC,1,58.75,12,1000,5400,66.75,57.75,58.50,3,58.50,70.00,74.38,S,1,58750.0
VNT,2,39.00,0,4500,4500,37.50,38.75,38.50,-1,38.75,39.00,43.50,I,4,175500.0
DIF,2,14.20,0,10000,60000,14.70,13.90,14.00,1,14.00,14.40,15.80,I,2,142000.0
KBANK,2,143.25,0,800,2400,136.50,138.00,142.00,8,142.50,153.00,170.00,I,4,114600.0
DCC,4,2.92,12,40000,40000,2.90,2.82,2.84,1,2.86,3.40,3.42,I,4,116800.0
IVL,4,44.00,12,3000,18000,42.00,42.75,43.00,1,43.25,50.00,54.38,I,4,132000.0
SAT,6,22.20,0,2500,7500,21.10,21.20,21.50,3,21.50,23.50,26.80,I,4,55500.0
RJH,7,34.75,0,1500,4500,33.00,32.50,33.00,2,33.00,39.00,36.20,I,4,52125.0
BGRIM,8,42.00,0,9000,9000,40.00,40.25,40.00,-1,40.75,56.00,54.00,I,4,378000.0


In [12]:
file_name = 'sell-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/sell-alert.csv',
 '\\Users\\User\\iCloudDrive\\sell-alert.csv',
 '\\Users\\User\\Dropbox\\sell-alert.csv')

In [13]:
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(output_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(data_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(box_file)